In [1]:
%load_ext autoreload
%autoreload 2
import time
from klampt import *
from klampt.math import vectorops,so3,se3
from klampt.io import loader
import math
import random
from IPython.display import clear_output,Markdown
import ipywidgets as widgets
import ipyklampt

world = WorldModel()
fn = "data/ur5Blocks.xml"
res = world.readFile(fn)
robot = world.robot(0)

# perturb the block configuration
objectTransforms = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]
random.shuffle(objectTransforms)
for i in range(world.numRigidObjects()):
    R,t = objectTransforms[i]
    t[0] += random.uniform(-0.05,0.05)
    t[1] += random.uniform(-0.05,0.05)
    R = so3.from_axis_angle(((0,0,1),random.uniform(0,math.pi*2)))
    world.rigidObject(i).setTransform(R,t)

#set the home configuration
qhome  = robot.getConfig()
qhome[1] = 2.3562
qhome[2] = -math.pi/3.0
qhome[3] = math.pi*2/3.0
qhome[4] = 0.64181
qhome[5] = math.pi/2.0

kvis = ipyklampt.KlamptWidget(world,width=600,height=400)
kvis.set_camera({u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.3093949571767591}, u'up': {u'y': 0.9488807720023056, u'x': 0.06236278376382235, u'z': -0.30941261080559734}, u'target': {u'y': 0.6599169321160283, u'x': -0.12092171623712519, u'z': 0.08376713045866419}})


In [2]:
#set up the controller and useful subroutines for converting to/from Klampt
def controller_to_klampt(qcontroller):
    """Converts a 6+1 controller configuration to Klampt format"""
    qorig = robot.getConfig()
    q = [v for v in qorig]
    for i in range(6):
        q[i+1] = qcontroller[i]
    robot.setConfig(q)
    robot.driver(6).setValue(qcontroller[6])
    q = robot.getConfig()
    robot.setConfig(qorig)
    return q

def klampt_to_controller(qklampt):
    """Converts a Klamp't configuration to 6+1 controller format"""
    qorig = robot.getConfig()
    res = [qklampt[i+1] for i in range(6)]
    robot.setConfig(qklampt)
    res.append(robot.driver(6).getValue())
    robot.setConfig(qorig)
    return res

CONTROLLER = 'simulation'
#when you are ready, replace the above line with this
#CONTROLLER = 'physical'
if CONTROLLER == 'simulation':
    from RobotControllerEmulator import kill_controller_threads,UR5WithGripperController
    controllerWorld = world.copy()
    kvis_ground_truth = ipyklampt.KlamptWidget(controllerWorld,width=600,height=400)
    robot_control_api = UR5WithGripperController(controllerWorld,randomize=True)
    VIS_UPDATE_FREQUENCY = 10
    VIS_UPDATE_ITERS = 0
    def _update_vis_hook():
        global VIS_UPDATE_ITERS
        VIS_UPDATE_ITERS += 1
        if VIS_UPDATE_ITERS % VIS_UPDATE_FREQUENCY == 0:
            #this is called inside the thread
            kvis_ground_truth.update()
    def start_controller():
        kill_controller_threads()
        robot_control_api.start(_update_vis_hook)
    kvis_ground_truth.set_camera({u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.3093949571767591}, u'up': {u'y': 0.9488807720023056, u'x': 0.06236278376382235, u'z': -0.30941261080559734}, u'target': {u'y': 0.6599169321160283, u'x': -0.12092171623712519, u'z': 0.08376713045866419}})
else:
    from robot_api.RobotController import UR5WithGripperController
    robot_control_api = UR5WithGripperController(host='192.168.0.1')
    def start_controller():
        robot_control_api.start()
def update_vis_from_controller():
    q = robot_control_api.getConfig()
    robot.setConfig(controller_to_klampt(q))
    kvis.update()
def stop_controller():
    robot_control_api.stop()
def wait_controller(duration):
    t0 = time.time()
    tstart = t0
    while time.time()-tstart < duration:
        update_vis_from_controller()
        t1 = time.time()
        time.sleep(max(0,0.05-(t1-t0)))
        t0 = t1


In [3]:
if CONTROLLER == 'simulation':
    display(Markdown("## Ground truth world"))
    display(kvis_ground_truth)

controls = widgets.HBox([])
display(controls)

display(Markdown("## Planning world"))
display(kvis)

output_area = widgets.Output()
display(output_area)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

## Ground truth world

S2xhbXB0V2lkZ2V0KGNhbWVyYT17dSdwb3NpdGlvbic6IHt1J3knOiAxLjkyOTg5ODM4NjY1NDQxMDgsIHUneCc6IDAuNzQ5ODI2OTIyMDUyNTM4NCwgdSd6JzogMS4zMDkzOTQ5NTcxNzY3NTnigKY=


HBox()

## Planning world

S2xhbXB0V2lkZ2V0KGNhbWVyYT17dSdwb3NpdGlvbic6IHt1J3knOiAxLjkyOTg5ODM4NjY1NDQxMDgsIHUneCc6IDAuNzQ5ODI2OTIyMDUyNTM4NCwgdSd6JzogMS4zMDkzOTQ5NTcxNzY3NTnigKY=


Output()

In [4]:
#Call this cell after you started the thread if you want to save CPU
stop_controller()

UR5WithGripperController emulation thread not started


In [5]:
#Call this cell to start the controller thread (the dot in the upper right of the notebook should flicker)
start_controller()
update_vis_from_controller()

Starting UR5WithGripperController emulation thread...
UR5GripperAPI thread #1 starting


In [6]:
#test control api
qbad = [0,0,0,0,0,0,0]
wait_controller(1)
q = robot_control_api.getConfig()
q[1] += .5
q[6] = 0
print "Setting first config"
robot_control_api.setConfig(q)
wait_controller(2)
q[1] -= .5
q[6] = 1
print "Setting second config"
robot_control_api.setConfig(qbad)
print "Waiting for 2 seconds"
wait_controller(2)
print "Done with movement"

Setting first config
Setting second config
Waiting for 2 seconds
Done with movement


In [7]:
#load camera and block marker poses from vision
#set up world to reflect sensor data

#IDs of objects in marker_poses.txt file
object_ids = [1,2,3]
id_to_object = dict((id,i) for (i,id) in enumerate(object_ids))
#known transforms from marker coordinates to object coordinates
marker_object_transforms = [(so3.identity(),(0,0,0.025))]*3

#load the calibrated camera transform
Tcamera = loader.load('RigidTransform','calibrated_camera_xform.txt')
print Tcamera

kvis.add_xform("CAMERA")
kvis.set_transform("CAMERA",*Tcamera)

#load the current markers from marker_poses.txt
def update_objects_from_sensors():
    object_poses = []
    with open('marker_poses.txt','r') as datafile:
        lines = datafile.readlines()
        for line in lines:
            if len(line.strip())==0:
                continue
            id,xform = line.split(None,1)
            T = loader.read('RigidTransform',xform)
            #this transpose is because the transform was written in column-major format, but loader assumed row-major
            T = (so3.inv(T[0]),T[1])
            object_poses.append((int(id),T))
    #print "Read object poses",object_poses
    #reset objects to be far away if they are not
    for i in xrange(world.numRigidObjects()):
        world.rigidObject(i).setTransform(*se3.identity())
    for (id,T) in object_poses:
        if id not in id_to_object:
            raise RuntimeError("Invalid object ID specified, must be one of "+str(object_ids))
        ind = id_to_object[id]
        Tobj_c = se3.mul(T,se3.inv(marker_object_transforms[ind]))
        Tobj = se3.mul(Tcamera,Tobj_c)
        world.rigidObject(ind).setTransform(*Tobj)

#emulate camera sensor readings by writing to marker_poses.txt
def emulate_camera_sensor():
    global world,marker_object_transforms,object_ids,Tcamera
    with open('marker_poses.txt','w') as datafile:
        for i in xrange(controllerWorld.numRigidObjects()):
            Tm_c = se3.mul(se3.inv(Tcamera),se3.mul(world.rigidObject(i).getTransform(),marker_object_transforms[i]))
            datafile.write(str(object_ids[i]) + '    ' + loader.write((so3.inv(Tm_c[0]),Tm_c[1]),'RigidTransform')+'\n')


([-0.0217268228463, -0.990605306817, 0.135015077952, -0.851489884476, -0.0524355829894, -0.521742739548, 0.523920720914, -0.126299785201, -0.842351139641], [0.105938110847, 0.114881052998, 1.07113035801])


In [8]:
numOfSamples = 25
maxIterations = 100

from klampt.model import ik
from klampt.model import collide
from klampt.plan import robotplanning
from klampt.robotsim import RobotModel
from klampt.model.trajectory import RobotTrajectory
from klampt.model.trajectory import Trajectory
from klampt.model.trajectory import SO3Trajectory
from klampt.model import trajectory
from klampt.math import vectorops
from klampt.robotsim import Geometry3D
from klampt.plan import robotplanning
from klampt.plan.robotplanning import SubsetMotionPlan
from klampt.plan.cspace import MotionPlan
from klampt.plan.robotcspace import RobotCSpace
import random
from klampt.model.collide import WorldCollider
from klampt.model import collide

pi = 3.14159
#information about the end effector
ee_link = 7
#the driver responsible for changing the gripper value
ee_driver = 6
#the position of the center link
ee_localpos = (0.15,0,0)
#the direction pointing from the palm toward the fingers
ee_fwd_dir = (1,0,0)
#the direction pointing perpendicular to the fingers
ee_perpendicular_dir = (0,0,1)

#driver parameters
ee_driver_open_value = 0.0
ee_driver_closed_value = 0.723

def closest_ik_solution(startConfig, endConfig, bottomLimit, topLimit):
    """TODO: complete me, if it helps
    In:
    - robot: a RobotModel instance
    - q: a candidate IK solution, which has joint angles in the range [-2pi,2pi]
    - qcurrent: the current configuration
    
    Out: a modified version of q that has the same joint angles modulo 2pi (hence, has the
    same layout in workspace), but is closer to qcurrent so that motion is minimized.
    """
    optimal_endConfig = []
    for i in range(len(endConfig)):
        optimal_endConfig.append(optimizeDimension(startConfig[i], endConfig[i], bottomLimit, topLimit))
    return optimal_endConfig
 
def optimizeDimension(startConfigAngle, endConfigAngle, bottomLimit, topLimit):
    currEndAngle = endConfigAngle
 
    minDistance = abs(startConfigAngle - currEndAngle)
    minEndAngle = currEndAngle
   
    while(currEndAngle + (2*math.pi) <= topLimit):
        currEndAngle += 2*math.pi
        currDistance = abs(startConfigAngle - currEndAngle)
        if (currDistance < minDistance):
            minDistance = currDistance
            minEndAngle = currEndAngle
    currEndAngle = endConfigAngle
 
    while(currEndAngle - (2*math.pi) >= bottomLimit):
        currEndAngle -= 2*math.pi
        currDistance = abs(startConfigAngle - currEndAngle)
        if (currDistance < minDistance):
            minDistance = currDistance
            minEndAngle = currEndAngle
    return minEndAngle
 
# print closest_ik_solution([2*math.pi,2*math.pi,0.0], [-2*math.pi,-2*math.pi,0.0], -2*math.pi, 2*math.pi)

def castIntegerListToFloatList(integerTuple):
    for i in range(len(integerTuple)):
        integerTuple[i] = float(integerTuple[i])
    return integerTuple

def create_samples(startPoint, endPoint, numOfSamples):        #returns a list of milestones along a straight line
    #cast startPoint and endPoint to float lists
    startPoint = castIntegerListToFloatList(startPoint)
    endPoint = castIntegerListToFloatList(endPoint)
    
    # configuration distance between start and end
    differenceTuple = vectorops.sub(endPoint, startPoint)
    
    # distance between each milestone
    mileStoneDistance = vectorops.div(differenceTuple, numOfSamples)
    
    # create milestones
    mileStones = []
    currPos = startPoint
    for i in range(0,numOfSamples+1):
        mileStones.append(currPos)
        currPos = vectorops.add(currPos, mileStoneDistance)
        
    #round milestone floats
    for i in range(len(mileStones)):
        for k in range(len(mileStones[i])):
            mileStones[i][k] = round(mileStones[i][k], 2)
    return mileStones

#Returns a new beginning point, the end point, and milestones from old beginning point to new beginning point
def otherOptions(robot, endPoint, maxIter, furthestWorkingMilestone):
    startPoint = furthestWorkingMilestone
    endPoint = endPoint
    
    #Create a random point
    newPoint = []
    for x in "12345678912345678":
        number = random.uniform(-3,3)
        newPoint.append(number)
    
    isVisible = visible(robot, startPoint, newPoint, numOfSamples, signal)
    
    for x in range(maxIter):
        if isVisible:
            return newPoint, endPoint, create_samples(startPoint, newPoint, numOfSamples)
        else:
            newPoint = []
            for x in "12345678912345678":
                number = random.uniform(-3,3)
                newPoint.append(number)
            isVisible = visible(robot, startPoint, newPoint, numOfSamples, signal)
    
    return "No feasible path exists between the endpoint and beginning with the number of max iterations given"
        
    #create new point to put in between start and endpoint
    #interpolate between startpoint and this new point

def betterInterpolator(robot, qstart, qend):
#     x = robotplanning.planToConfig( world,
#                                     robot,
#                                     qend)
#                                     edgeCheckResolution = 1e-2,
#                                     extraConstraints = [],
#                                     equalityConstraints = [],
#                                     equalityTolerance = 1e-3,
#                                     ignoreCollisions = [],   #world.rigidObject(object_dropdown.value)
#                                     movingSubset = 'auto')
    x = robotplanning.makeSpace(world,
                                robot,
                                edgeCheckResolution = 1e-2,
                               )
#     y = SubsetMotionPlan.getPath(robot, qstart, qend)
    z = robotplanning.planToSet(world, robot, x)
    b = RobotCSpace.interpolate(x, qstart, qend, 1e-2)
#     a = MotionPlan.getPath(robot, milestone1 = qstart, milestone2 = qend)
    print ("x is", x)
#     print y
    print ("z is", z)
#     print a
    print ("b is", b)
    
def masterInterpolator(robot, qstart, qend, samplingRate, signal):
    #where the master list of milestones will be kept
    milestones = []
    [boolean, partialStones] = visible(robot, qstart, qend, samplingRate, signal)
    if (boolean == True):
        for milestone in partialStones:
            milestones.append(milestone)
        return milestones
    if (boolean == False):
        return "There is no feasible path"
        for milestone in partialStones:
            milestones.append(milestone)
        [newBeginning, End, newStones] = otherOptions(robot, qend, maxIterations, partialStones[-1])  
        for milestone in newStones:
            milestones.append(newStones)
        masterInterpolator(robot, newBeginning, qend, numOfSamples, signal)     
    return milestones

def feasible(robot,q, signal):
    """
    In:
    - robot: a RobotModel instance
    - q: a configuration
    Out: True if q is feasible (collision free, in joint limits)"""
    robot.setConfig(q)
    #x is a robotCSpace instance
    x = robotplanning.makeSpace(world,
                                robot,
                                edgeCheckResolution = 1e-2,
                                )
    print ("Does the robot self collide?", RobotCSpace.selfCollision(x, x=None))
    if RobotModel.selfCollides(robot):
        return False
    if RobotCSpace.selfCollision(x, x=None):
        return False

    collider = collide.WorldCollider(world)
    
#     transforms = []
#     counter = 0
#     for x in objectTransforms:
#         counter += 1
#         transforms[counter] = x
#     geometryOfBlock = WorldCollider.getGeom(collider, world.rigidObject(object_dropdown.value).getTransform())
#     print "GEOMTRY OF BLOCK IS", geometryOfBlock


    #NEED TO CREATE RIGID OBJECT MODELS FOR EACH BLOCK WITH THE INPUT OF THEIR TRANSFORMS (objectTransforms[x])
    
    #WorldCollider.ignoreCollision(collider, #RIGIDOBJECTMODEL)
    
    if (signal == True):
        b = any(WorldCollider.robotObjectCollisions(collider, robot))
        print ("Does the robot collide with Environment?", b)
        if b:
            return False
    
    
#     if RobotCSpace.envCollision(x, x=None):
#         return False

#     qmin,qmax = robot.getJointLimits()
#     for i, q in enumerate(robot.getConfig()):
#         if q < qmin[i] or q > qmax[i]:
#             print "There is a joint limit error"
#             return False
    return True
    
def visible(robot,qstart,qend, samplingRate, signal):
    """In:
    - robot: a RobotModel instance
    - qstart,qend: start and end configurations
    Out: True if the straight line joint space path between qstart / qend is feasible
         Return a list of the milestones that work"""
    milestones = create_samples(qstart, qend, samplingRate)
    workingMilestones = []
    for i in range(len(milestones)):
        isValid = feasible(robot, milestones[i], signal)
        if isValid:
            workingMilestones.append(milestones[i])  
        if (not isValid):
            print "There is no valid linear path between the start and end configurations, implement CSpace motion planning to get a nonlinear path"
            return [False, workingMilestones]
    return [True, milestones]

def set_gripper_width(robot,q,width):
    """Helper: produces a configuration which is the same as q, except that the
    gripper is opened to the given width (in the range 0 (closed) to 1 (open))"""
    qold = robot.getConfig()
    robot.setConfig(q)
    robot.driver(ee_driver).setValue(ee_driver_open_value*width + (1.0-width)*ee_driver_closed_value)
    qres = robot.getConfig()
    robot.setConfig(qold)
    return qres

qbad = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
def move_home(robot):
    """
    In:
    - robot: a RobotModel instance

    Out: a list of milestones interpolating from the robot's current configuration to the home configuration.
      None can be returned if your planner cannot find a solution.
    """
    signal = True
    global qhome
    state = 'HOME'
    qstart = RobotModel.getConfig(robot)
    qhome = closest_ik_solution(qstart, qhome, -2*pi, 2*pi)
    qterrible = []
    for x in "12345678912345678":
        number = random.uniform(-3,3)
        qterrible.append(number)
#     print betterInterpolator(robot, qstart, qhome)
    return masterInterpolator(robot, qstart, qhome, numOfSamples, signal)

def find_pregrasp_config(robot,block_pose,offset_height=0.06):
    """
    TODO: complete me for C.1
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at qhome
    - block_pose: a transform of the center of the block
    - offset_height: the distance to hover above the block
    
    Out: a configuration q reaching the target pregrasp.
      None can be returned if there is no solution, but in all test cases there should be a solution.
    """
    rot = so3.mul(block_pose[0], so3.from_axis_angle(((0.0, 1.0, 0.0), math.pi/2)))
    trans = (block_pose[1][0], block_pose[1][1], block_pose[1][2] + ee_localpos[0]/2 + block_width + offset_height)
    obj = ik.objective(robot.link(ee_link), ref=None, R=rot, t=trans) 
    ik.solve(obj)
    return set_gripper_width(robot,robot.getConfig(),1)

def move_pregrasp(robot,block_pose,offset_height=0.06):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at qhome
    - block_pose: a transform of the center of the block
    - offset_height: the distance to hover above the block
    
    Out: a list of milestones interpolating from the robot's current configuration to the target pregrasp.
      None can be returned if your planner cannot find a solution.
    """
    qStart = robot.getConfig()
    signal = True
    R = block_pose[0]
    RMultiplier = so3.from_axis_angle([[0,1,0], pi/2])
    R = so3.mul(R, RMultiplier)
    
    t = block_pose[1]
    number = .15 + .06                                    #this is equivalent to the length of the end effector + the .06 that we're required to do
    a = [0,0, number]
    t = vectorops.add(t,a)

    obj = ik.objective(robot.link(7), R=R, t=t)
    ik.solve(obj)
    qFinal = robot.getConfig()
    qGrasp = set_gripper_width(robot,qFinal,1)
    qGrasp = closest_ik_solution(qStart, qGrasp, -2*pi, 2*pi)
    return masterInterpolator(robot, qStart, qGrasp, numOfSamples, signal)


def lower_and_close(robot,block_pose,close_width):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at the pregrasp configuration
    - block_pose: a transform of the center of the block
    - close_width: the amount that the gripper should be closed
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving straight down (in Cartesian space), and then closing the robot's gripper.
    
    None can be returned if your planner cannot find a solution.
    """
    signal = False
    killMe = 8000
    print killMe
    actualSamplingRate = numOfSamples / 4
    milestones = []
    
    qStart = robot.getConfig()
    R = block_pose[0]   
    RMultiplier = so3.from_axis_angle([[0,1,0], pi/2])
    R = so3.mul(R, RMultiplier)
    t = block_pose[1]
    vector = [0,0, +.15]
    t = vectorops.add(t, vector)
    
    obj = ik.objective(robot.link(7), R=R, t=t)
    ik.solve(obj)
    qDown = robot.getConfig()

    OBJ1 = masterInterpolator(robot, qStart, qDown, actualSamplingRate, signal)
    
    qClose = set_gripper_width(robot,qDown,close_width)
    qClose = closest_ik_solution(qStart, qClose, -2*pi, 2*pi)
    
    qClose = klampt_to_controller(qDown)
    qClose[6] = 3.0
    qClose = controller_to_klampt(qClose)
    
    OBJ2 = masterInterpolator(robot, qDown, qClose, actualSamplingRate, signal)
    
    signal = True
    
    for x in OBJ1:
        milestones.append(x)
    for x in OBJ2:
        milestones.append(x)
    return milestones
    
def lift_and_place(robot,target_position):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently gripping an object
    - target_position: the position of the target box
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving 1) to lift the block up, 2) to the target position, and 3) opening the gripper
    
    None can be returned if your planner cannot find a solution.
    """
    signal = False
    killMe = 8000000
    print killMe
    milestones = []
    actualSamplingRate = numOfSamples / 3
    
    qstart = robot.getConfig()
    link = robot.link(ee_link)
    R, t = link.getTransform()
    deltaT = [0,0,.1]
    t = vectorops.add(t, deltaT)
    
    obj1 = ik.objective(robot.link(7), R = R, t = t)
    ik.solve(obj1)
    
    qup = robot.getConfig()
    qup = closest_ik_solution(qstart, qup, -2*pi, 2*pi)
#     sampleGroup1 = create_samples(qstart, qup, actualSamplingRate)
    sampleGroup1 = masterInterpolator(robot, qstart, qup, actualSamplingRate, signal)
    R2, t2 = link.getTransform()
    obj2 = ik.objective(robot.link(7), R = R2, t = target_position)
    ik.solve(obj2)
    qbox = robot.getConfig()
    qbox = closest_ik_solution(qup, qbox, -2*pi, 2*pi)
#     sampleGroup2 = create_samples(qup, qbox, actualSamplingRate)
    sampleGroup2 = masterInterpolator(robot, qup, qbox, actualSamplingRate, signal)
    qopen = set_gripper_width(robot,qbox,1)
    qopen = closest_ik_solution(qbox, qopen, -2*pi, 2*pi)
#     sampleGroup3 = create_samples(qbox, qopen, actualSamplingRate)
    sampleGroup3 = masterInterpolator(robot, qbox, qopen, actualSamplingRate, signal)
    for x in sampleGroup1:
        milestones.append(x)
    for x in sampleGroup2:
        milestones.append(x)
    for x in sampleGroup3:
        milestones.append(x)
    signal = True 
#     print milestones
    return milestones
#     return [qstart,qup,qbox,qopen]


In [9]:
#run this cell to bind your code to the visualization playback window. You should 
#not need to modify this code, unless you want to tweak the trajectory execution speeds

from klampt.model.trajectory import RobotTrajectory
from klampt.model import trajectory
import RobotControllerEmulator
from klampt.robotsim import *
from klampt.robotsim import RobotModel


#target parameters
target_position = (0.26,-0.15,1.2)

#block parameters
block_width = 0.05
block_names = [world.rigidObject(i).getName() for i in range(world.numRigidObjects())]
block_poses = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]

#gripper parameters
block_close_amount = 0.5

#the speed of trajectory execution: default 0.5 radian per second
#this may be tweaked, if desired
trajectory_execution_speed = 0.5

#create controller

#state can be 'HOME', 'PREGRASP', 'GRASPED', 'BOX'
state = 'HOME'

kvis.add_text("HUD1",1,1)
kvis.add_text("HUD2",1,5)
kvis.add_text("HUD3",1,9)
kvis.add_text("HUD4",1,13)
kvis.add_text("HUD5",1,17)
kvis.add_text("HUD6",1,21)
kvis.add_text("HUD7",1,25)

#the amount of time between refreshes
dt = 0.05
                
def execute_trajectory(traj):
    """Executes a trajectory on the controller.  You may or may not want to modify this."""
    global robot,controls
    for c in controls.children:
        c.disabled = True
        
    #send the trajectory
    dtraj = traj.discretize(dt)
    q0 = robot_control_api.getConfig()
    q0_traj = klampt_to_controller(traj.milestones[0])
    robot_control_api.setConfig(klampt_to_controller(traj.milestones[0]))
    
    t0 = traj.times[0]
    for t,q in zip(dtraj.times[1:],dtraj.milestones[1:]):
        wait_controller(t-t0)
        robot_control_api.setConfig(klampt_to_controller(q))
#         if(not feasible(robot,q)):
#             return kill()
        t0 = t
    
    for c in controls.children:
        c.disabled = False


def trajectory_from_path(path):
    """TODO: perform a smoother path interpolation"""
    global robot
    times = [0.0]
    
    maxvel = RobotModel.getVelocityLimits(robot)
    maxacc = RobotModel.getAccelerationLimits(robot)
    
    #timing = L2 -> faster overall movement, doesn't report overreaching limits
    #timing = limited -> slower overall movement, reports overreaching limits sometimes
    traj = trajectory.path_to_trajectory(path, 
                                         velocities = 'trapezoidal', #minimum-jerk
                                         timing = 'L2', #L2, #limited
                                         smoothing = 'spline', 
                                         zerotol = None,
                                         vmax = 3.15,
                                         amax = 10,
                                         speed = 1.0,
                                         dt = 0.01,
                                         startvel = 0,
                                         endvel = 0
                                         )
    milestones = traj.milestones
    times = traj.times
    print traj
    return traj

def execute_path(path):
    execute_trajectory(trajectory_from_path(path))
    
def do_move_home(button):
    global state,robot,qhome
    output_area.clear_output()
    with output_area:
        try:
            
            milestones = move_home(robot)
            print("MOVE HOME milestonesLength", len(milestones))
#             print("MOVE HOME milestones", milestones)
            if milestones is not None:
                execute_path(milestones)
        except Exception:
            import traceback
            traceback.print_exc()
    state = 'HOME'

def do_sense(button):
    output_area.clear_output()
    with output_area:
        if CONTROLLER == 'simulation':
            emulate_camera_sensor()
        update_objects_from_sensors()
    kvis.update()
    
def do_move_pregrasp(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'HOME':
                print "WARNING: should do pregrasp from the home configuration"
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            selected_object = object_dropdown.value
            milestones = move_pregrasp(robot,world.rigidObject(selected_object).getTransform())
            print("MOVE PREGRASP milestonesLength", len(milestones))
#             print("MOVE PREGRASP milestones", milestones)
            if milestones is not None:  # Piazza Fix
                execute_path(milestones)
                state = 'PREGRASP'
        except Exception:
            import traceback
            traceback.print_exc()
        
            
def do_grasp(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'PREGRASP':
                print "WARNING: should do grasp from the pregrasp configuration"
            selected_object = object_dropdown.value
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            milestones = lower_and_close(robot,world.rigidObject(selected_object).getTransform(),block_close_amount)
            print(" DO GRASP milestonesLength", len(milestones))
#             print("DO GRASP milestones", milestones)
            if milestones is not None:
                execute_path(milestones)
                state = 'GRASPED'
        except Exception:
            import traceback
            traceback.print_exc()

def do_place(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'GRASPED':
                print "WARNING: should do place from the grasped configuration"
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            milestones = lift_and_place(robot,target_position)
            print("DO PLACE milestonesLength", len(milestones))
#             print("DO PLACE milestones", milestones)
            if milestones is not None:
                execute_path(milestones)
                state = 'BOX'
        except Exception:
            import traceback
            traceback.print_exc()
        
object_dropdown = widgets.Dropdown(
    options=block_names,
    value=block_names[0],
    description='Object:'
)

move_home_button = widgets.Button(description="Move Home")
sense_button = widgets.Button(description="Sense")
move_pregrasp_button = widgets.Button(description="Move to Pregrasp")
grasp_button = widgets.Button(description="Grasp")
place_button = widgets.Button(description="Place")
move_home_button.on_click(do_move_home)
sense_button.on_click(do_sense)
move_pregrasp_button.on_click(do_move_pregrasp)
grasp_button.on_click(do_grasp)
place_button.on_click(do_place)

controls.children = (move_home_button,sense_button,object_dropdown,move_pregrasp_button,grasp_button,place_button)

## Written answers

Problem B

Problem C

In [10]:
"""
C) For part B we first focused on creating milestones for each of the movements, and then focused on creating
trajectories out of those milestones.  For the first part we used parts of our code from Lab 2.  We then created
helper functions that would take in a starting qconfiguration, and ending qconfiguration, and a sampling rate 
determined by the user, and would return a list of milestones along the linear trajectory from start to end
of length equal to the sampling rate.  However, if any of the milestones were invalid, either due to world or
self collisions, or joint constraints, the function would return null.  This is to prevent the robot from hitting
anything or breaking.  If all the milestones are valid, then linear interpolation is (probably, given a
high enough sampling rate) possible.  The list of milestones is then returned, or appended to with more milestones
in more complex movements such as lift_and_place

To create trajectories out of the milestones, we examined Hermitian, Trapezoidal, and Cosine smoothing.  We decided
to move forward with trapezoidal smoothing, as it would quickly use maximum acceleration to reach the maximum 
velocity, and the time needed to go from maximum velocity to zero velocity is constant throughout all the milestone
interpolations if we use maximum acceleration.  While we began to implement this function using the kinematic equations
like velocity = v(o) + a(t^2), and d = v(o)t + .5(a)(t^2), we eventually found a Klampt function that would execute trapezoidal 
smoothing given milestone and other inputs.  The function took into account our concerns over making sure the robot
didn't start and stop with zero velocity at every milestone (which would look terrible), obeying max velocity and
max acceleration values which we found for each joint in the robot to be roughly similar, and using L2 timing to
not overreach any of these limits.  The trapezoidal profile usually uses maximum acceleration values to go from the 
start velocity to reach the maximum velocity.  It then changes its acceleration to smooth to 0 once the maximum 
velocity has been reached, and stay there until a calculated time at which the robot begins to decelerate to reach
the desired end velocity at the milestone.  Our beginning and end velocities were 0, and the time to decelerate was
calculated to ensure that the end velocity (0) was a function of the maximum deceleration values, maximum velocity/operating velocity,
and time.  The function implemented trapezoidal smoothing with our robot's parameters and our specified velocities,
and was able to make a smooth looking transition between the  milestones that were passed into the function.
Overall for Part C we created a list of milestones that could dynamically change dependent on the sampling rate passed in
and used those milestones to implement smoothing, specifically using a function that implemented trapezoidal smoothing
based off parameters we defined and also found from our robot.
"""

"\nC) For part B we first focused on creating milestones for each of the movements, and then focused on creating\ntrajectories out of those milestones.  For the first part we used parts of our code from Lab 2.  We then created\nhelper functions that would take in a starting qconfiguration, and ending qconfiguration, and a sampling rate \ndetermined by the user, and would return a list of milestones along the linear trajectory from start to end\nof length equal to the sampling rate.  However, if any of the milestones were invalid, either due to world or\nself collisions, or joint constraints, the function would return null.  This is to prevent the robot from hitting\nanything or breaking.  If all the milestones are valid, then linear interpolation is (probably, given a\nhigh enough sampling rate) possible.  The list of milestones is then returned, or appended to with more milestones\nin more complex movements such as lift_and_place\n\nTo create trajectories out of the milestones, we exa

In [11]:
"""
B) For Part B we focused on recreating Lab 2 movement with the caveat of checking that
the paths between start and end configurations were all collision free.  To do this, we 
implemented several helper functions.  We created a function that optimized qconfigurations
to optimize movement between qconfigurations, and created a function that create_samples 
that created milestones between the start and end configurations.  The number of milestones
created is based off of a dynamic samplingRate parameter that is specified by the user, and
we found in Part C that a samplingRate of between 20 - 50 provided optimal results, while also
testing enough configurations to hold a high confidence that the entire path was problem-free.
In testing for collisions we tested for self collisions using CSpace methods, as well as 
world collisions using WorldModel.  We created a WorldModel instance and edited it so that
touching the blocks would not be counted as a collision.  We also took into account joint limits.
While we did not implement ensuring the object doesn't collide with anything once it has been picked up 
to work dynamically, we instead created a high enough movement around the box to prevent it for 
our Lab's purposes since we know the box height.  However, to work in multiple situations, we would 
first ignore the block's geometries, and then at each milestone test to see if the block's 
geometries were colliding with any of the world's geometries.  This can theoretically be 
accomplished using the Geometry3D package, but we had toruble implementing this functionality,
and so did not implement a way to check if the block's geometries collided with the world 
geometries.  

Currently, our work for part B only checks if a path between start and end configurations is possible based off of
world collisions, self collisions, and joint limits.  However, in the case that the path is not feasible, the robot
will refuse to move to the end point.  We laid some groundwork for a plan to let the robot get around obstacles
and create new paths so that the robot will still move to the endpoint, but take a non-colliding route to get there.
This pleans were started in betterInterpolator and otherOptions, in which we tried to move to the furthest working
milestone, and then create a new milestone and have the robot move there (if the path to it is feasible) and then
rerunning a linear interpolation pattern to try to achieve the end goal.  This proved extremely difficult given the
time constraints for this project and difficulties in implementing methods in the API, and so we were not able to
fully implement this functionality.  Given more time, we would use the groundwork in those two methods to fully
enable this functionality.



"""

"\nB) For Part B we focused on recreating Lab 2 movement with the caveat of checking that\nthe paths between start and end configurations were all collision free.  To do this, we \nimplemented several helper functions.  We created a function that optimized qconfigurations\nto optimize movement between qconfigurations, and created a function that create_samples \nthat created milestones between the start and end configurations.  The number of milestones\ncreated is based off of a dynamic samplingRate parameter that is specified by the user, and\nwe found in Part C that a samplingRate of between 20 - 50 provided optimal results, while also\ntesting enough configurations to hold a high confidence that the entire path was problem-free.\nIn testing for collisions we tested for self collisions using CSpace methods, as well as \nworld collisions using WorldModel.  We created a WorldModel instance and edited it so that\ntouching the blocks would not be counted as a collision.  We also took int